In [3]:
def compute_shapes(x_2Dshape, kernel, stride, padding):
    x_h, x_w = x_2Dshape
    s_h, s_w = stride
    k_h, k_w = kernel
    
    if padding == "valid":
        y_h = int(np.ceil((x_h - k_h + 1) / s_h))
        y_w = int(np.ceil((x_w - k_w + 1) / s_w))
        zeros_h, zeros_w = (0,0)
    elif padding == "same":
        y_h = int(np.ceil(x_h / s_h))
        y_w = int(np.ceil(x_w / s_w))

        # (y-1): index of last number of y, (y-1)*s: mapped to index in x
        # (y-1)*s+k: count of needed elements of x, like (y-1)*s+1 + k-1 
        # (y-1)*s+k-x: count of extra elements from x, possibly negative when x is enough to cover
        zeros_h = max((y_h - 1) * s_h + k_h - x_h, 0)
        zeros_w = max((y_w - 1) * s_w + k_w - x_w, 0)
    else:
        raise ValueError("Incorrect padding type.")
        

    new_h = x_h + zeros_h
    new_w = x_w + zeros_w
        
    shapes = ((new_h, new_w), (y_h, y_w))
    return shapes

def pad(X, new_x_shape):

    n_in, x_h, x_w = X.shape
    new_h, new_w = new_x_shape
    
    if (new_h, new_w) == (x_h, x_w): # no padding
        return X

    zeros_h = new_h - x_h
    zeros_w = new_w - x_w
    pad0 = zeros_h // 2
    pad1 = zeros_h - pad0
    pad2 = zeros_w // 2
    pad3 = zeros_w - pad2
    
    padded_X = np.zeros(shape=(n_in, new_h, new_w))
    padded_X[:, pad0:-pad1, pad2:-pad3] = X
        
    return padded_X

def depad(g_X, old_x_shape):
    
    old_xh, old_xw = old_x_shape
    g_X = g_X[:, :old_xh, :old_xw]  

    return g_X

In [4]:
import numpy as np
import random
from IPython.core.debugger import set_trace

class Conv2D:
                     
    def __init__(self, n_in, n_out, kernel, 
                 padding="same", stride=(1,1), W=None, b=None):
        
        self.n_in = n_in
        self.n_out = n_out
        assert kernel != None and len(kernel) == 2
        self.kernel = kernel

        assert padding == "valid" or padding == "same"
        self.padding = padding 
        self.stride = stride

        if W is None:
            k_h, k_w = kernel
            size = n_out * n_in * k_h * k_w 
            weights = np.random.uniform(size=size).reshape(n_out, n_in, k_h, k_w)
            scale = np.sqrt(2./size)
            W = weights * scale       
        if b is None:
            b = np.zeros(n_out)
       
        self.W = W
        self.b = b
        self.X = None
        self.Y = None
        self.padded_X_shape = None
        self.X_shape = None
        self.Y_shape = None
        
        self.g_W = None
        self.g_b = None
        self.g_X = None
        self.g_padded_X = None
        self.g_Y = None
        
        return
           
    def forward(self, X):
    
        assert X.ndim == 3
        assert (np.array(X[0].shape) > np.array(self.kernel)).all()

        self.X = X
        if self.X_shape is None:
            self.X_shape = X.shape
            n_in, x_h, x_w = X.shape
            padded_x_shape, y_shape = self.compute_shapes((x_h, x_w))
            self.padded_X_shape = (n_in, *padded_x_shape)
            self.Y_shape = (self.n_out, *y_shape)
        else:
            assert X.shape == self.X_shape

        self.X = pad(X, self.padded_X_shape[1:])
        Y = np.zeros(shape=self.Y_shape)
                
        y_h = self.Y_shape[1]
        y_w = self.Y_shape[2]
        k_h, k_w = self.kernel
        s_h, s_w = self.stride

        for i_Y in range(self.n_out):
            for i_X in range(self.n_in):
                k = self.W[i_Y,i_X]
                x = self.X[i_X]
                for i in range(y_h):
                    for j in range(y_w):
                        r = i * s_h
                        c = j * s_w
                        Y[i_Y, i, j] += (x[r:r+k_h, c:c+k_w] * k).sum()
             
            Y[i_Y] += self.b[i_Y]
        
        self.Y = Y
        return self.Y
    
    def backward(self, g_Y):
        
        assert g_Y.shape == self.Y_shape
        n_out, y_h, y_w = self.Y_shape

        s_h, s_w = self.stride
        k_h, k_w = self.kernel

        self.g_Y = g_Y
        g_W = np.zeros(shape=(n_out, self.n_in, k_h, k_w))
        g_b = np.zeros(n_out)
        

        for i_Y in range(self.n_out):
            g_y = self.g_Y[i_Y]
            for i_X in range(self.n_in):
                x = self.X[i_X]
                for i_kh in range(k_h):
                    for i_kw in range(k_w):
                        hs = i_kh; he = hs + y_h * s_h
                        ws = i_kw; we = ws + y_w * s_w
                        g_W[i_Y,i_X,i_kh,i_kw] = (g_y * x[hs:he:s_h, ws:we:s_w]).sum()
        
            g_b[i_Y] = g_y.sum()

        g_padded_X = np.zeros(shape=self.padded_X_shape)

        for i_yh in range(y_h):
            i_xh = i_yh * s_h
            for i_yw in range(y_w):
                i_xw = i_yw * s_w
                for i_Y in range(self.n_out):
                    g_padded_X[:, i_xh:i_xh+k_h, i_xw:i_xw+k_w] += self.W[i_Y] * g_Y[i_Y, i_yh, i_yw]  
            
        self.g_Y = g_Y
        self.g_W = g_W
        self.g_b = g_b
        self.g_padded_X = g_padded_X
        self.g_X = depad(g_padded_X, self.X_shape[1:])
        
        return self.g_X
    
    def update(self, g_Y, learning=0.1):
        
        self.W -= learning * self.g_W
        self.b -= learning * self.g_b
        
    def compute_shapes(self, x_2Dshape):
        return compute_shapes(x_2Dshape, self.kernel, self.stride, self.padding)
            
    def __str__(self):
        
        s = "\nX is:" + ("" if self.X is None else str(self.X.shape))
        s += "\n" + str(self.X)
        s += "\nX_shape: " + str(self.X_shape) +  "padded_X_shape: " + str(self.padded_X_shape) 
        s += "\nY is:" + ("" if self.Y is None else str(self.Y.shape))
        s += "\n" + str(self.Y)
        s += "\nkernel is: " + str(self.W.shape)
        s += "\n" + str(self.W)
        s += "\nbias is: " + str(self.b.shape)
        s += "\n" + str(self.b)
        s += "\nstride is:\n" + str(self.stride)
        
        s += "\ng_Y is:" + ("" if self.g_Y is None else str(self.g_Y.shape))
        s += "\n" + str(self.g_Y)
        s += "\ng_W is:" + ("" if self.g_W is None else str(self.g_W.shape))
        s += "\n" + str(self.g_W)
        s += "\ng_b is:" + ("" if self.g_b is None else str(self.g_b.shape))
        s += "\n" + str(self.g_b)
        s += "\ng_X is:" + ("" if self.g_X is None else str(self.g_X.shape))
        s += "\n" + str(self.g_X)
        return s

# when we set all the kernels, biases, g_Y as ones, the resulted g_X shows 
# how many times one input x element has been used in forwarding

In [5]:
class Pooling:
    def __init__(self, kernel=(2,2), stride=(2,2), pool="MAX"):
        self.padding = "same"

        self.name = pool
        self.kernel = kernel
        self.stride = stride
        
        self.X = None
        self.Y = None
        self.padded_X_shape = None
        self.X_shape = None
        self.Y_shape = None
        self.g_X = None
        self.g_Y = None
        return
        
    def forward(self, X):
        self.X = X
        if self.X_shape is None:
            n_in = X.shape[0]
            self.X_shape = X.shape
            padded_x_shape, y_shape = self.compute_shapes(X.shape[1:])
            self.padded_X_shape = (n_in, *padded_x_shape)
            self.Y_shape = (n_in, *y_shape)
        else:
            assert X.shape == self.X_shape

        self.X = pad(X, self.padded_X_shape[1:])
        Y = np.empty(shape=self.Y_shape)
        n, yh, yw = self.Y_shape
        sh, sw = self.stride
        kh, kw = self.kernel

        for i_n in range(n):
            for i_yh in range(yh):
                for i_yw in range(yw):
                    r0 = i_yh*sh; r1 = r0+kh
                    c0 = i_yw*sw; c1 = c0+kw
                    Y[i_n, i_yh, i_yw] = X[i_n, r0:r1, c0:c1].max()
        
        self.Y = Y  
        return Y
    
    def backward(self, g_Y):
        assert g_Y.shape == self.Y_shape
        self.g_Y = g_Y
        
        g_padded_X = np.zeros(shape=self.padded_X_shape)
        n, yh, yw = self.Y_shape
        sh, sw = self.stride
        kh, kw = self.kernel

        for i_n in range(n):
            for i_yh in range(yh):
                for i_yw in range(yw):
                    y = self.Y[i_n, i_yh, i_yw]
                    g_y = g_Y[i_n, i_yh, i_yw]
                
                    r0 = i_yh*sh; r1 = r0+kh
                    c0 = i_yw*sw; c1 = c0+kw
                    g_padded_X[i_n, r0:r1, c0:c1] += g_y * (self.X[i_n, r0:r1, c0:c1]==y)
    
        self.g_padded_X = g_padded_X
        self.g_X = depad(g_padded_X, self.X_shape[1:])
        return self.g_X
    
    def compute_shapes(self, x_2Dshape):
        return compute_shapes(x_2Dshape, self.kernel, self.stride, self.padding)

    def __str__(self):
        s = "\n Pooling is: " + self.name 
        s += "\nX is:" + ("" if self.X is None else str(self.X.shape))
        s += "\n" + str(self.X)
        s += "\nY is:" + ("" if self.Y is None else str(self.Y.shape))
        s += "\n" + str(self.Y)
        s += "\ng_Y is:" + ("" if self.g_Y is None else str(self.g_Y.shape))
        s += "\n" + str(self.g_Y)
        s += "\ng_X is:" + ("" if self.g_X is None else str(self.g_X.shape))
        s += "\n" + str(self.g_X)
        
        return s

In [6]:
class ConvolutionLayer:
    def __init__(self, conv, acti=None, pool=None): 
        if conv is None: raise ValueError("Convolution layer is not assigned.")
        self.conv = conv

        if acti is not None:
            self.acti = Activation(acti)
            self.A = None
        else:
            self.acti = None

        
        self.pool = pool

        self.X = None
        self.Y = None
        self.Y_shape = None
        return
    
    def forward(self, X):
        self.X = X
        self.Y = self.conv.forward(X)
        
        if self.acti is not None:
            self.A = self.acti.func(self.Y)
            self.Y = self.A
        if self.pool is not None:
            self.Y = self.pool.forward(self.Y)
        
        if self.Y_shape is None:
            self.Y_shape = self.Y.shape
        else:
            assert self.Y_shape == self.Y.shape
        
        return self.Y
    
    def backward(self, g_Y):
        self.g_Y = g_Y
    
        if self.pool is not None:
            g_Y = self.pool.backward(g_Y)
        if self.acti is not None:
            g_acti = self.acti.grad(self.A)
            g_Y = g_acti * g_Y
            
        g_Y = self.conv.backward(g_Y)
        return g_Y
        
    def update(self, learning):
        self.conv.update(learning)
        return 
    
    def compute_shapes(self, x_2Dshape):
        
        x_2Dshape = self.conv.compute_shapes(x_2Dshape)[1]
        if self.pool is not None:
            x_2Dshape = self.pool.compute_shapes(x_2Dshape)[1]
        
        return x_2Dshape
    
    def OutputToFC(self):
        return self.Y.reshape(-1, 1)

    def InputFromFC(self, g_Y):
        return g_Y.reshape(self.Y_shape)
    
    def __str__(self):
        pass

In [7]:
class CNN:
    def __init__(self, learning=0.001):
        
        self.type = "CNN"
        self.learning = learning
        
        self.convlayers = []

        n_in = 1; n_out = 5; kernel = (5,5); stride = (3,3); padding="same"
        c1 = Conv2D(n_in, n_out, kernel=kernel, padding=padding, stride=stride)
        convlayer1 = ConvolutionLayer(c1, acti="RELU", pool=None)
        self.convlayers.append(convlayer1)

        x_shape = (28, 28)
        y_h, y_w = convlayer1.compute_shapes(x_shape)  
        
        # this layer can be skipped
        # -------------
        # n_in = n_out; n_out = 50; kernel = (3,3); stride = (1,1); padding="same"
        # c2 = Conv2D(n_in, n_out, kernel=kernel, padding=padding, stride=stride)
        # convlayer2 = ConvolutionLayer(c2, acti="RELU", pool=None)
        # self.convlayers.append(convlayer2)
        # x_shape = (y_h, y_w)
        # y_h, y_w = convlayer2.compute_shapes(x_shape)
        # -------------

        n_in = n_out * y_h * y_w; n_out = 100
        f = PercepLayer(n_in, n_out, acti="RELU")
        self.perceplayer = f
        
        n_in = n_out; n_out = 10
        f = SoftMaxLayer(n_in, n_out)
        self.outlayer = f
        return
    
    def forward(self, X):
        for convlayer in self.convlayers:
            X = convlayer.forward(X)
            
        X = convlayer.OutputToFC()
        X = self.perceplayer.forward(X)
        return self.outlayer.forward(X)
    
    def backward(self, label):
        g_Y = self.outlayer.backward(label)
        g_Y = self.perceplayer.backward(g_Y)
        g_Y = self.convlayers[-1].InputFromFC(g_Y)
        
        for convlayer in self.convlayers[-1::-1]:
            g_Y = convlayer.backward(g_Y)
        return
    
    def update(self, learning):
        self.outlayer.update(learning)
        self.perceplayer.update(learning)
        for convlayer in self.convlayers:
            convlayer.update(learning)

        return
    
    def train_1sample(self, X, label):
        self.forward(X)
        self.backward(label)
        self.update(self.learning)
        return
 
    def predict_1sample(self, X):
        predict = self.forward(X)
        return predict


In [ ]:
%%capture
%run 'multilayer-perceptron.ipynb'

def is_main_module():
    return __name__ == '__main__' and '__file__' not in globals()

In [ ]:
def run_cnn_mnist():
    #set_trace()
    cnn = CNN()
    mnist = MNIST(cnn)
    for i in range(5):
        mnist.train(-1)
        accuracy = mnist.test(-1)
        print("\nEpoch {} accuracy: {}".format(i, accuracy))

if is_main_module():
    np.seterr(all='raise')
    run_cnn_mnist()
            


Epoch 0 accuracy: 0.9541
